<a href="https://colab.research.google.com/github/tewei0328/teach-database/blob/main/sqlite2_Billionaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3

df_1 = pd.read_csv('https://raw.githubusercontent.com/tewei0328/data/master/Billionaire.csv')  #讀取CSV資料集檔案
 
conn = sqlite3.connect('billionaire.db')  #建立資料庫
cursor = conn.cursor()
cursor.execute('CREATE TABLE Billionaire(Name, NetWorth, Country, Source, Rank, Age, Industry)')  #建立資料表
conn.commit()

df_1.head()

#如果資料表存在，就寫入資料，否則建立資料表
df_1.to_sql('Billionaire', conn, if_exists='append', index=False)

#透過SQL語法讀取資料庫中的資料
us_df = pd.read_sql("SELECT * FROM Billionaire WHERE Country='United States'", conn)
print(us_df)

                      Name NetWorth        Country              Source  Rank  \
0               Jeff Bezos   $177 B  United States              Amazon     1   
1                Elon Musk   $151 B  United States       Tesla, SpaceX     2   
2               Bill Gates   $124 B  United States           Microsoft     4   
3          Mark Zuckerberg    $97 B  United States            Facebook     5   
4           Warren Buffett    $96 B  United States  Berkshire Hathaway     6   
..                     ...      ...            ...                 ...   ...   
719             Vlad Tenev     $1 B  United States       stock trading  2674   
720  Vlad Vendrow & family     $1 B  United States            software  2674   
721        J. Wayne Weaver     $1 B  United States               Shoes  2674   
722            Sandy Weill     $1 B  United States           Citigroup  2674   
723             Yu De-Chao     $1 B  United States     pharmaceuticals  2674   

      Age               Industry  
0   

## [Day 14] 回測分析
### 從零開始的套牢生活 - AI股票預測系統 系列 第 14 篇
https://ithelp.ithome.com.tw/m/articles/10267871?fbclid=IwAR0GdoJVPIJ6VfcSdAfJzWAgHoaYahPsyBVIAY9NzjhNIfmmWHfsKYJC7Os

# 引入函式庫

In [ ]:
# ! pip install backtesting
import datetime
import warnings

import pandas as pd
import requests
#from backtesting import Backtest, Strategy
#from backtesting.lib import crossover
#from backtesting.test import SMA
#from pandas.core.common import SettingWithCopyWarning

#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pd.set_option("display.max_columns", None)


# 取得資料
backtesting強制要求欄位名稱格式，
所以需要先轉換欄位名稱

https://finmindtrade.com/analysis/#/data/document

## 1-股票

In [ ]:
stock_index = '0050'
url = "https://api.finmindtrade.com/api/v4/data"
parameter = {
    "dataset": "TaiwanStockPrice",
    "start_date": datetime.datetime(2023, 4, 1, 0, 0).strftime("%Y-%m-%d"),
    "end_date": datetime.datetime(2023, 5, 13, 0, 0).strftime("%Y-%m-%d"),
    "data_id": stock_index,
}

data = requests.get(url, params=parameter)
data = data.json()



In [ ]:
data

{'msg': 'success',
 'status': 200,
 'data': [{'date': '2023-04-06',
   'stock_id': '0050',
   'Trading_Volume': 8076540,
   'Trading_money': 975648621,
   'open': 120.85,
   'max': 121.0,
   'min': 120.2,
   'close': 121.0,
   'spread': -0.6,
   'Trading_turnover': 15029},
  {'date': '2023-04-07',
   'stock_id': '0050',
   'Trading_Volume': 4623088,
   'Trading_money': 559443394,
   'open': 121.25,
   'max': 121.45,
   'min': 120.8,
   'close': 121.0,
   'spread': 0.0,
   'Trading_turnover': 6781},
  {'date': '2023-04-10',
   'stock_id': '0050',
   'Trading_Volume': 4503497,
   'Trading_money': 545831612,
   'open': 121.25,
   'max': 121.65,
   'min': 120.9,
   'close': 121.0,
   'spread': 0.0,
   'Trading_turnover': 10059},
  {'date': '2023-04-11',
   'stock_id': '0050',
   'Trading_Volume': 6233410,
   'Trading_money': 753799317,
   'open': 121.0,
   'max': 121.15,
   'min': 120.4,
   'close': 120.85,
   'spread': -0.15,
   'Trading_turnover': 7741},
  {'date': '2023-04-12',
   'stoc

In [ ]:
df = pd.DataFrame(data["data"])

df.index = pd.to_datetime(df["date"])
df.rename(
    columns={
        "Trading_Volume": "Volume",
        "open": "Open",
        "max": "High",
        "min": "Low",
        "close": "Close",
    },
    inplace=True,
)

df.drop(
    columns=["stock_id", "date", "Trading_money", "spread", "Trading_turnover"],
    inplace=True,
)

df

,Volume,Open,High,Low,Close
date,,,,,
2023-04-06,8076540,120.85,121.00,120.20,121.00
2023-04-07,4623088,121.25,121.45,120.80,121.00
2023-04-10,4503497,121.25,121.65,120.90,121.00
2023-04-11,6233410,121.00,121.15,120.40,120.85
2023-04-12,3247078,120.85,120.85,120.15,120.55
2023-04-13,7509494,120.05,120.15,119.50,119.50
2023-04-14,3289954,120.00,120.90,119.85,120.65
2023-04-17,6061114,120.35,120.50,119.80,120.35
2023-04-18,5183750,120.35,120.40,119.75,119.95


In [ ]:
df.query('Close>120')

,Volume,Open,High,Low,Close
date,,,,,
2023-04-06,8076540,120.85,121.00,120.20,121.00
2023-04-07,4623088,121.25,121.45,120.80,121.00
2023-04-10,4503497,121.25,121.65,120.90,121.00
2023-04-11,6233410,121.00,121.15,120.40,120.85
2023-04-12,3247078,120.85,120.85,120.15,120.55
2023-04-14,3289954,120.00,120.90,119.85,120.65
2023-04-17,6061114,120.35,120.50,119.80,120.35


In [ ]:
df.query('Close<117')

,Volume,Open,High,Low,Close
date,,,,,
2023-04-25,21039325,118.30,118.30,116.55,116.6
2023-04-26,14209529,116.55,116.60,115.65,116.1
2023-04-27,10274310,116.05,116.65,115.50,116.0


In [ ]:
df.query('Close<117 or Close>120')

,Volume,Open,High,Low,Close
date,,,,,
2023-04-06,8076540,120.85,121.00,120.20,121.00
2023-04-07,4623088,121.25,121.45,120.80,121.00
2023-04-10,4503497,121.25,121.65,120.90,121.00
2023-04-11,6233410,121.00,121.15,120.40,120.85
2023-04-12,3247078,120.85,120.85,120.15,120.55
2023-04-14,3289954,120.00,120.90,119.85,120.65
2023-04-17,6061114,120.35,120.50,119.80,120.35
2023-04-25,21039325,118.30,118.30,116.55,116.60
2023-04-26,14209529,116.55,116.60,115.65,116.10


In [ ]:
df.query('Close>117 and Close<118')

,Volume,Open,High,Low,Close
date,,,,,
2023-04-28,6023403,116.85,117.55,116.85,117.40
2023-05-02,5572977,117.45,118.00,117.10,117.70
2023-05-03,4652271,117.35,117.35,116.80,117.20
2023-05-04,3888293,117.00,117.50,116.70,117.50
2023-05-05,2550129,117.70,118.05,117.40,117.80
2023-05-10,4685154,118.80,118.80,117.65,117.80
2023-05-11,4031551,118.25,118.45,117.55,117.55
2023-05-12,4251764,117.15,117.50,116.75,117.20


## FinMind API 使用說明
https://finmindtrade.com/analysis/#/data/api

## 選擇權日夜盤成交資訊，包含到期月份、開高低收、漲跌幅、交易量、結算價格。
https://finmindtrade.com/analysis/#/data/document

## 2-選擇權

In [ ]:
stock_index = "TXO"
url = "https://api.finmindtrade.com/api/v4/data"
parameter = {
    "dataset": "TaiwanOptionDaily",
    "start_date": datetime.datetime(2023, 4, 1, 0, 0).strftime("%Y-%m-%d"),
    "end_date": datetime.datetime(2023, 5, 13, 0, 0).strftime("%Y-%m-%d"),
    "data_id": stock_index,
}

data = requests.get(url, params=parameter)
data = data.json()


In [ ]:
data

In [ ]:
df = pd.DataFrame(data["data"])
df.head()

,date,option_id,contract_date,strike_price,call_put,open,max,min,close,volume,settlement_price,open_interest,trading_session
0,2023-04-06,TXO,202304W1,13900.0,call,0.0,0.0,0.0,0.0,0,0.0,1,position
1,2023-04-06,TXO,202304W1,13900.0,call,0.0,0.0,0.0,0.0,0,0.0,0,after_market
2,2023-04-06,TXO,202304W1,13900.0,put,0.0,0.0,0.0,0.0,0,0.0,156,position
3,2023-04-06,TXO,202304W1,13900.0,put,0.0,0.0,0.0,0.0,0,0.0,0,after_market
4,2023-04-06,TXO,202304W1,14000.0,call,0.0,0.0,0.0,0.0,0,0.0,0,position


In [ ]:
df = pd.DataFrame(data["data"])

df.index = pd.to_datetime(df["date"])
df.rename(
    columns={
        "open": "Open",
        "max": "High",
        "min": "Low",
        "close": "Close",
    },
    inplace=True,
)

df.drop(
    #columns=["option_id", "date", "settlement_price", "open_interest", "trading_session"],
    columns=["option_id", "date", "settlement_price", "open_interest"],
    inplace=True,
)

df

,contract_date,strike_price,call_put,Open,High,Low,Close,volume,trading_session
date,,,,,,,,,
2023-04-06,202304W1,13900.0,call,0.0,0.0,0.0,0.0,0,position
2023-04-06,202304W1,13900.0,call,0.0,0.0,0.0,0.0,0,after_market
2023-04-06,202304W1,13900.0,put,0.0,0.0,0.0,0.0,0,position
2023-04-06,202304W1,13900.0,put,0.0,0.0,0.0,0.0,0,after_market
2023-04-06,202304W1,14000.0,call,0.0,0.0,0.0,0.0,0,position
...,...,...,...,...,...,...,...,...,...
2023-05-12,202312,19000.0,put,0.0,0.0,0.0,0.0,0,after_market
2023-05-12,202312,19200.0,call,0.0,0.0,0.0,0.0,0,position
2023-05-12,202312,19200.0,call,13.0,13.0,7.1,7.1,5,after_market


In [ ]:
def get_options(option_df, contract_period, put_or_call, strike_price, trade_period):
    
    option_df = df.loc[(df['contract_date'] == contract_period) & \
                       (df['call_put'] == put_or_call) & \
                       (df['strike_price'] == strike_price) & \
                       (df['trading_session'] == trade_period) ]
    return option_df

In [ ]:
get_options(df, '202305', 'call', 16000, 'position')

,contract_date,strike_price,call_put,Open,High,Low,Close,volume,trading_session
date,,,,,,,,,
2023-04-06,202305,16000.0,call,237.0,237.0,194.0,204.0,189,position
2023-04-07,202305,16000.0,call,227.0,239.0,211.0,218.0,97,position
2023-04-10,202305,16000.0,call,225.0,259.0,217.0,226.0,282,position
2023-04-11,202305,16000.0,call,207.0,267.0,207.0,246.0,218,position
2023-04-12,202305,16000.0,call,253.0,253.0,216.0,235.0,203,position
2023-04-13,202305,16000.0,call,195.0,215.0,175.0,178.0,521,position
2023-04-14,202305,16000.0,call,197.0,250.0,186.0,226.0,955,position
2023-04-17,202305,16000.0,call,209.0,224.0,192.0,224.0,627,position
2023-04-18,202305,16000.0,call,205.0,209.0,171.0,176.0,1022,position


## 3-期貨

## 期貨日成交資訊
https://finmindtrade.com/analysis/#/data/api

In [ ]:
stock_index = "TX"
url = "https://api.finmindtrade.com/api/v4/data"
parameter = {
    "dataset": "TaiwanFuturesDaily",
    "start_date": datetime.datetime(2023, 4, 1, 0, 0).strftime("%Y-%m-%d"),
    "end_date": datetime.datetime(2023, 5, 13, 0, 0).strftime("%Y-%m-%d"),
    "data_id": stock_index,
}

data = requests.get(url, params=parameter)
data = data.json()


In [ ]:
data

In [ ]:
df = pd.DataFrame(data["data"])
df.head()

,date,futures_id,contract_date,open,max,min,close,spread,spread_per,volume,settlement_price,open_interest,trading_session
0,2023-04-06,TX,202304,15780.0,15793.0,15701.0,15760.0,-104.0,-0.66,71495,15762.0,68514,position
1,2023-04-06,TX,202304,15848.0,15928.0,15810.0,15919.0,55.0,0.35,39855,0.0,0,after_market
2,2023-04-06,TX,202305,15793.0,15793.0,15678.0,15737.0,-100.0,-0.63,1362,15737.0,4392,position
3,2023-04-06,TX,202305,15808.0,15902.0,15785.0,15891.0,54.0,0.34,501,0.0,0,after_market
4,2023-04-06,TX,202306,15700.0,15700.0,15615.0,15670.0,-101.0,-0.64,172,15670.0,1221,position


In [ ]:
def get_futures(option_df, contract_period, trading_session):
    
    option_df = df.loc[(df['contract_date'] == contract_period) & \
                       (df['trading_session'] == trading_session) ]
    return option_df

In [ ]:
get_futures(df, '202305', 'position')

,date,futures_id,contract_date,open,max,min,close,spread,spread_per,volume,settlement_price,open_interest,trading_session
2,2023-04-06,TX,202305,15793.0,15793.0,15678.0,15737.0,-100.0,-0.63,1362,15737.0,4392,position
22,2023-04-07,TX,202305,15807.0,15842.0,15772.0,15791.0,54.0,0.34,748,15793.0,4498,position
38,2023-04-10,TX,202305,15805.0,15898.0,15802.0,15823.0,30.0,0.19,1322,15823.0,4933,position
57,2023-04-11,TX,202305,15810.0,15931.0,15801.0,15897.0,74.0,0.47,1903,15894.0,5536,position
77,2023-04-12,TX,202305,15922.0,15929.0,15838.0,15895.0,1.0,0.01,1361,15895.0,5777,position
97,2023-04-13,TX,202305,15830.0,15873.0,15782.0,15791.0,-104.0,-0.65,2619,15790.0,7132,position
118,2023-04-14,TX,202305,15855.0,15967.0,15828.0,15923.0,133.0,0.84,6341,15922.0,10412,position
140,2023-04-17,TX,202305,15896.0,15952.0,15856.0,15949.0,27.0,0.17,30321,15949.0,27635,position
163,2023-04-18,TX,202305,15911.0,15937.0,15846.0,15871.0,-78.0,-0.49,35258,15872.0,48482,position
186,2023-04-19,TX,202305,15876.0,15907.0,15768.0,15778.0,-94.0,-0.59,56290,15778.0,64317,position


# 回測策略
如果快線超過慢線，表示股票進入上漲的趨勢，此時進行買空操作，分析圖表以![https://ithelp.ithome.com.tw/upload/images/20210919/20141586vhDmTPQp9Z.png](https://ithelp.ithome.com.tw/upload/images/20210919/20141586vhDmTPQp9Z.png)表示。\
如果快線跌過慢線，表示股票進入下跌的趨勢，此時進行賣空操作，分析圖表以![https://ithelp.ithome.com.tw/upload/images/20210919/20141586Y98Ntg07AE.png](https://ithelp.ithome.com.tw/upload/images/20210919/20141586Y98Ntg07AE.png)表示。\
綠色代表上一次操作的獲利情形，綠色表示獲利，綠色表示賠錢。

In [ ]:
class SmaCross(Strategy):
    def init(self):
        self.fast_line = self.I(SMA, self.data.Close, 20)
        self.slow_line = self.I(SMA, self.data.Close, 50)

    def next(self):
        if crossover(self.fast_line, self.slow_line):
            print(
                f"{self.data.index[-1]} Buy: Price: {self.data.Close[-1]}, Slow: {self.slow_line[-5:]}, Fast: {self.fast_line[-5:]}"
            )
            self.buy()
        elif crossover(self.slow_line, self.fast_line):
            print(
                f"{self.data.index[-1]} Sell: Price: {self.data.Close[-1]}, Slow: {self.slow_line[-5:]}, Fast: {self.fast_line[-5:]}"
            )

            self.sell()


In [ ]:
class SmaCross1(Strategy):
# 只多不空  
    def init(self):
        self.fast_line = self.I(SMA, self.data.Close, 20)
        self.slow_line = self.I(SMA, self.data.Close, 50)

    def next(self):
        if crossover(self.fast_line, self.slow_line):
            print(
                f"{self.data.index[-1]} Buy: Price: {self.data.Close[-1]}, Slow: {self.slow_line[-5:]}, Fast: {self.fast_line[-5:]}"
            )
            #self.buy()
            # If ema1 crosses above ema2, buy the asset
            #self.position.close()
            self.buy()
        elif crossover(self.slow_line, self.fast_line):
            print(
                f"{self.data.index[-1]} Sell: Price: {self.data.Close[-1]}, Slow: {self.slow_line[-5:]}, Fast: {self.fast_line[-5:]}"
            )
            self.position.close()
            #self.sell()

# 執行策略
- cash：本金。
- commission：交易費用。
- exclusive_orders：是否禁止多頭操作(同時只能買空或賣空)。
- trade_on_close：於收盤時交易，否則預設於開盤交易。

exclusive_orders：若為True時，每次操作前自動關閉(close)上次操作。
Buy(500)、Buy(1000) -> 最後持股1000
Buy(500)、Sell(500) -> 賣空500
.
如果trade_on_close是True，市價訂單將根據當前柱的收盤價而不是下一個柱的開盤價執行。

如果hedging是True，則允許同時雙向交易。如果False，則反向訂單首先以FIFO方式關閉現有交易。

如果exclusive_orders是True，每個新訂單都會自動關閉之前的交易/頭寸，每次最多進行一次有效的交易（多頭或空頭）。

In [ ]:
test = Backtest(
    df,
    SmaCross1,
    cash=1000000,
    commission=0.004,
    exclusive_orders=True,
    trade_on_close=True,
)
result = test.run()


2015-04-17 00:00:00 Sell: Price: 142.5, Slow: [146.86 146.92 146.98 147.02 146.97], Fast: [147.725 147.475 147.325 147.225 146.675]
2015-05-18 00:00:00 Buy: Price: 146.5, Slow: [147.44 147.39 147.33 147.25 147.17], Fast: [146.725 146.925 147.075 147.05  147.25 ]
2015-05-27 00:00:00 Sell: Price: 145.0, Slow: [147.05 147.08 147.11 147.08 147.  ], Fast: [147.725 147.65  147.4   147.15  146.825]
2015-10-07 00:00:00 Buy: Price: 135.0, Slow: [128.86 128.72 128.65 128.57 128.59], Fast: [127.525 127.85  128.125 128.525 129.15 ]
2016-01-11 00:00:00 Sell: Price: 133.0, Slow: [140.45 140.38 140.28 140.23 140.16], Fast: [141.475 141.125 140.75  140.4   140.075]
2016-02-18 00:00:00 Buy: Price: 151.0, Slow: [139.78 139.83 140.   140.18 140.35], Fast: [137.95  138.5   139.225 139.975 140.85 ]
2016-05-06 00:00:00 Sell: Price: 148.0, Slow: [155.56 155.55 155.47 155.4  155.34], Fast: [157.175 156.45  155.875 155.575 155.25 ]
2016-06-13 00:00:00 Buy: Price: 162.0, Slow: [154.36 154.41 154.49 154.62 154.6

# 分析結果
將分析結果儲存，並利用預設瀏覽器開啟，
result為詳細的分析結果。

In [ ]:
test.plot()
print(result)

Start                     2015-01-05 00:00:00
End                       2023-05-05 00:00:00
Duration                   3042 days 00:00:00
Exposure Time [%]                   96.753566
Equity Final [$]                  1577378.174
Equity Peak [$]                   2758836.636
Return [%]                          57.737817
Buy & Hold Return [%]              258.422939
Return (Ann.) [%]                    5.812039
Volatility (Ann.) [%]                25.17699
Sharpe Ratio                         0.230847
Sortino Ratio                        0.355037
Calmar Ratio                         0.134267
Max. Drawdown [%]                  -43.287321
Avg. Drawdown [%]                   -6.434191
Max. Drawdown Duration     1572 days 00:00:00
Avg. Drawdown Duration      111 days 00:00:00
# Trades                                   35
Win Rate [%]                        28.571429
Best Trade [%]                      99.876171
Worst Trade [%]                    -13.989794
Avg. Trade [%]                    

In [ ]:
#test.plot(filename=f"./backtest_result/{stock_index}.html")
#print(result)

![https://ithelp.ithome.com.tw/upload/images/20210919/20141586nTsggfN69g.png](https://ithelp.ithome.com.tw/upload/images/20210919/20141586nTsggfN69g.png)


欄位 | 說明
------------- | -------------
Start                  |   起始時間
End                    |   結束時間
Duration               |   經過天數
Exposure [%]           |   投資比率
Equity Final [$]       |   最終資產
Equity Peak [$]        |   最高資產
Return [%]             |   報酬率
Buy & Hold Return [%]  |   買入持有報酬率
Max. Drawdown [%]      |   最大交易回落
Avg. Drawdown [%]      |   平均交易回落
Max. Drawdown Duration |   最長交易回落期間
Avg. Drawdown Duration |   平均交易回落期間
Win Rate [%]           |   勝率
Best Trade [%]         |   最好交易報酬率
Worst Trade [%]        |   最差交易報酬率
Avg. Trade [%]         |   平均交易報酬率
Max. Trade Duration    |   最長交易間隔
Avg. Trade Duration    |   平均交易間隔
Expectancy [%]         |   期望值
SQN                    |   系統品質指標
Sharpe Ratio           |   夏普比率
Sortino Ratio          |   索丁諾比率
Calmar Ratio           |   卡瑪比率
_strategy              |   使用策略名稱
*From https://hackmd.io/@s02260441/SkA7IWVJv*
